I want to clean data and analyze the top 3-5 diseases throughout months and years. 

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import os

%matplotlib inline

In [5]:
# Read the csv file
df18 = pd.read_csv("18_NNDSS.csv")
df18.head()

,Disease,MMWR year,MMWR week,Current week,"Current week, flag",Cum 2018,"Cum 2018, flag",5-year weekly average§,"5-year weekly average§, flag",Total cases reported for pervious years 2017,"Total cases reported for pervious years 2017, flag",Total cases reported for pervious years 2016,"Total cases reported for pervious years 2016, flag",Total cases reported for pervious years 2015,"Total cases reported for pervious years 2015, flag",Total cases reported for pervious years 2014,"Total cases reported for pervious years 2014, flag",Total cases reported for pervious years 2013,"Total cases reported for pervious years 2013, flag",States reporting cases during current week (No.)
0,Anthrax,2018.0,1.0,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN
1,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,1.0,NaN,-,NaN,-,6.0,NaN,115.0,NaN,247.0,NaN,896.0,NaN,NaN,NN,NaN,NN,NaN
2,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,1.0,NaN,-,NaN,-,0.0,NaN,3.0,NaN,7.0,NaN,6.0,NaN,8.0,NaN,8.0,NaN,NaN
3,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,1.0,NaN,-,NaN,-,0.0,NaN,67.0,NaN,15.0,NaN,11.0,NaN,11.0,NaN,22.0,NaN,NaN
4,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,1.0,NaN,-,NaN,-,NaN,-,44.0,NaN,35.0,NaN,55.0,NaN,80.0,NaN,85.0,NaN,NaN


In [6]:
# Clean up the column headers
df18.columns = df18.columns.str.strip().str.lower().str.replace('  ', ' ').str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [7]:
# Delete unnecessary columns
columns_to_drop = ['current_week', 'current_week,_flag', 'cum_2018', 'cum_2018,_flag', '5-year_weekly_average§', 
                   '5-year_weekly_average§,_flag', 'total_cases_reported_for_pervious_years_2017,_flag', 
                   'total_cases_reported_for_pervious_years_2016,_flag', 
                   'total_cases_reported_for_pervious_years_2015,_flag', 
                   'total_cases_reported_for_pervious_years_2014,_flag', 
                   'total_cases_reported_for_pervious_years_2013,_flag', 
                   'states_reporting_cases_during_current_week_no.']

df18.drop(columns_to_drop, inplace=True, axis=1)

In [8]:
# Delete non ASCII characters
df18['disease'] = df18['disease'].apply(lambda x: ''.join([" " if (ord(i) < 32 or ord(i) > 126) or ord(i) == 42  else i for i in x]))

In [9]:
df18.head()

,disease,mmwr_year,mmwr_week,total_cases_reported_for_pervious_years_2017,total_cases_reported_for_pervious_years_2016,total_cases_reported_for_pervious_years_2015,total_cases_reported_for_pervious_years_2014,total_cases_reported_for_pervious_years_2013
0,Anthrax,2018.0,1.0,NaN,NaN,NaN,NaN,NaN
1,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,1.0,115.0,247.0,896.0,NaN,NaN
2,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,1.0,3.0,7.0,6.0,8.0,8.0
3,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,1.0,67.0,15.0,11.0,11.0,22.0
4,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,1.0,44.0,35.0,55.0,80.0,85.0


In [10]:
# Shorten some of the column names
df18.columns = df18.columns.str.replace('_reported_for_pervious_years_', '_')

In [11]:
df18.head()

,disease,mmwr_year,mmwr_week,total_cases_2017,total_cases_2016,total_cases_2015,total_cases_2014,total_cases_2013
0,Anthrax,2018.0,1.0,NaN,NaN,NaN,NaN,NaN
1,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,1.0,115.0,247.0,896.0,NaN,NaN
2,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,1.0,3.0,7.0,6.0,8.0,8.0
3,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,1.0,67.0,15.0,11.0,11.0,22.0
4,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,1.0,44.0,35.0,55.0,80.0,85.0


In [12]:
df18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3393 entries, 0 to 3392
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   disease           3393 non-null   object 
 1   mmwr_year         3392 non-null   float64
 2   mmwr_week         3392 non-null   float64
 3   total_cases_2017  2345 non-null   float64
 4   total_cases_2016  2332 non-null   float64
 5   total_cases_2015  2491 non-null   float64
 6   total_cases_2014  2385 non-null   float64
 7   total_cases_2013  2225 non-null   float64
dtypes: float64(7), object(1)
memory usage: 212.2+ KB


In [13]:
# Replace null values with 0
cols_to_ints = ['mmwr_year', 'mmwr_week', 'total_cases_2017', 'total_cases_2016', 'total_cases_2015', 
                'total_cases_2014', 'total_cases_2013']


df18[cols_to_ints] = df18[cols_to_ints].fillna(0)
#astype(int)

In [14]:
df18.isnull().sum()

disease             0
mmwr_year           0
mmwr_week           0
total_cases_2017    0
total_cases_2016    0
total_cases_2015    0
total_cases_2014    0
total_cases_2013    0
dtype: int64

In [15]:
df18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3393 entries, 0 to 3392
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   disease           3393 non-null   object 
 1   mmwr_year         3393 non-null   float64
 2   mmwr_week         3393 non-null   float64
 3   total_cases_2017  3393 non-null   float64
 4   total_cases_2016  3393 non-null   float64
 5   total_cases_2015  3393 non-null   float64
 6   total_cases_2014  3393 non-null   float64
 7   total_cases_2013  3393 non-null   float64
dtypes: float64(7), object(1)
memory usage: 212.2+ KB


In [16]:
# Check for duplicate rows
sum(df18.duplicated())

58

In [17]:
has_duplicate = df18.duplicated()

duplicates = df18[has_duplicate]

duplicates

,disease,mmwr_year,mmwr_week,total_cases_2017,total_cases_2016,total_cases_2015,total_cases_2014,total_cases_2013
2048,"Meningococcal disease, Unknown serogroup",2018.0,32.0,131.0,142.0,120.0,196.0,298.0
2050,Streptococcal toxic-shock syndrome,2018.0,32.0,372.0,283.0,335.0,259.0,224.0
2051,"Hepatitis B, perinatal infection",2018.0,32.0,31.0,34.0,37.0,47.0,48.0
2052,"Arboviral diseases, neuroinvasive and nonneur...",2018.0,32.0,5.0,7.0,6.0,8.0,8.0
2054,"Botulism, total",2018.0,32.0,177.0,201.0,195.0,161.0,152.0
2055,Chancroid,2018.0,32.0,7.0,11.0,11.0,0.0,0.0
2056,Vancomycin-resistant Staphylococcus aureus,2018.0,32.0,0.0,0.0,3.0,0.0,0.0
2057,Plague,2018.0,32.0,1.0,4.0,16.0,10.0,4.0
2058,"Botulism, infant",2018.0,32.0,137.0,144.0,138.0,127.0,136.0
2059,"Haemophilus influenzae, invasive disease, age ...",2018.0,32.0,189.0,159.0,135.0,266.0,233.0


In [18]:
#Week 32 has duplicate data, need to drop duplicates
df18.drop_duplicates(inplace = True)

sum(df18.duplicated())

0

In [19]:
# Filter out diseases with 0 cases between 2013-2017
df18 = df18[(df18.total_cases_2017 > 0) | (df18.total_cases_2016 > 0) | (df18.total_cases_2015 > 0)| (df18.total_cases_2014 > 0) & (df18.total_cases_2013 > 0)]

In [20]:
df18.shape

(2449, 8)

In [21]:
# Average all of the cases for the years
df18 = df18.groupby('disease').mean()
df18
#astype(int)

,mmwr_year,mmwr_week,total_cases_2017,total_cases_2016,total_cases_2015,total_cases_2014,total_cases_2013
disease,,,,,,,
"Arboviral diseases , Chikungunya virus",2018.0,34.000000,156.000000,247.0,896.0,0.0,0.0
"Arboviral diseases , Eastern equine encephalitis virus",2018.0,34.000000,5.000000,7.0,6.0,8.0,8.0
"Arboviral diseases , Jamestown Canyon virus",2018.0,34.000000,75.000000,15.0,11.0,11.0,22.0
"Arboviral diseases , La Crosse virus",2018.0,34.000000,63.000000,35.0,55.0,80.0,85.0
"Arboviral diseases , Powassan virus",2018.0,34.000000,34.000000,22.0,7.0,8.0,12.0
...,...,...,...,...,...,...,...
Vancomycin-resistant Staphylococcus aureus,2018.0,26.352941,0.352941,0.0,3.0,0.0,0.0
Vancomycin-resistant Staphylococcus aureus,2018.0,34.000000,109.000000,108.0,183.0,212.0,248.0
"Viral hemorrhagic fever , Crimean-Congo hemorrhagic fever",2018.0,34.000000,0.000000,0.0,3.0,0.0,0.0


In [22]:
# Add a column for total cases
df18['case_total'] = df18['total_cases_2017'] + df18['total_cases_2016'] + df18['total_cases_2015'] + df18['total_cases_2014'] + df18['total_cases_2013']
df18

,mmwr_year,mmwr_week,total_cases_2017,total_cases_2016,total_cases_2015,total_cases_2014,total_cases_2013,case_total
disease,,,,,,,,
"Arboviral diseases , Chikungunya virus",2018.0,34.000000,156.000000,247.0,896.0,0.0,0.0,1299.000000
"Arboviral diseases , Eastern equine encephalitis virus",2018.0,34.000000,5.000000,7.0,6.0,8.0,8.0,34.000000
"Arboviral diseases , Jamestown Canyon virus",2018.0,34.000000,75.000000,15.0,11.0,11.0,22.0,134.000000
"Arboviral diseases , La Crosse virus",2018.0,34.000000,63.000000,35.0,55.0,80.0,85.0,318.000000
"Arboviral diseases , Powassan virus",2018.0,34.000000,34.000000,22.0,7.0,8.0,12.0,83.000000
...,...,...,...,...,...,...,...,...
Vancomycin-resistant Staphylococcus aureus,2018.0,26.352941,0.352941,0.0,3.0,0.0,0.0,3.352941
Vancomycin-resistant Staphylococcus aureus,2018.0,34.000000,109.000000,108.0,183.0,212.0,248.0,860.000000
"Viral hemorrhagic fever , Crimean-Congo hemorrhagic fever",2018.0,34.000000,0.000000,0.0,3.0,0.0,0.0,3.000000
